In [41]:
import pandas as pd
import os
import glob
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [42]:
path=r'C:\Users\stang\Google Drive\Data_for_fun\Fantasy Basketball\DailyGameLogs'
allFiles = glob.glob(path + "/*.csv")

gamelogs = pd.concat([pd.read_csv(f, encoding='cp1252') for f in glob.glob('*.csv')])

In [43]:
path1=r'C:\Users\stang\Google Drive\Data_for_fun\Fantasy Basketball'

gamelogs.to_csv(os.path.join(path1,r'GameLogs.csv'))

In [44]:
gamelogs['Date'] = pd.to_datetime(gamelogs['Date'])
gamelogs.sort_values(by=['Player','Date'], inplace=True, ascending=False) 
# Want latest game to be game 1 - easy filter for last 10 games
gamelogs['Game_Num'] = gamelogs.groupby('Player').cumcount() + 1

### Full Season

In [45]:
data_group = gamelogs.groupby(by=["Player"]).agg({'GmSc':'size','PTS':['sum'],'TRB':['sum'],'BLK':['sum'], 'STL':['sum'], \
                                              'AST':['sum'], '3P':['sum'], 'FG':['sum'], 'FGA':['sum'], 'FT':['sum'], \
                                              'FTA':['sum'], 'TOV':['sum']})

In [46]:
data_group["PTS_G"] = data_group["PTS", "sum"] / data_group["GmSc", "size"]
data_group["TRB_G"] = data_group["TRB", "sum"] / data_group["GmSc", "size"]
data_group["BLK_G"] = data_group["BLK", "sum"] / data_group["GmSc", "size"]
data_group["STL_G"] = data_group["STL", "sum"] / data_group["GmSc", "size"]
data_group["AST_G"] = data_group["AST", "sum"] / data_group["GmSc", "size"]
data_group["3P_G"] = data_group["3P", "sum"] / data_group["GmSc", "size"]

data_group["FGA"] = np.where(data_group["FGA"] == 0, 1, data_group["FGA"])
data_group["FG%"] = data_group["FG", "sum"]**2 / data_group["GmSc", "size"]
# data_group["FGA", "sum"]

data_group["FTA"] = np.where(data_group["FTA"] == 0, 1, data_group["FTA"])
data_group["FT%"] = data_group["FT", "sum"]**2 / data_group["FTA", "sum"] 
# data_group["FTA", "sum"]

data_group["TOV_G"] = data_group["TOV", "sum"] / data_group["GmSc", "size"]

In [47]:
column_names_to_normalize = ['PTS_G', 'TRB_G', 'BLK_G', 'STL_G', 'AST_G', '3P_G', 'FG%', 'FT%', 'TOV_G']
x = data_group[column_names_to_normalize].values
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data_group.index)
data_group[column_names_to_normalize] = df_temp

In [48]:
data_group["sum"] = data_group["PTS_G"] + data_group["TRB_G"] + data_group["BLK_G"] + data_group["STL_G"] + data_group["AST_G"] + \
                    data_group["3P_G"] + data_group["FG%"] + data_group["FT%"] - data_group["TOV_G"] 
data_group = data_group.sort_values(by=['sum'], ascending=False)

In [49]:
data_group1 = data_group.loc[:, ['PTS_G', 'TRB_G', 'BLK_G', 'STL_G', 'AST_G', '3P_G', 'FG%', 'FT%', 'TOV_G', 'sum']]

In [50]:
team = pd.read_csv(r'C:\Users\stang\Google Drive\Data_for_fun\Fantasy Basketball\FBteam.csv', encoding='cp1252')

In [51]:
team_stats = data_group1[data_group1.index.isin(team["Player"])]
total = team_stats.sum(axis=0)

In [52]:
team_stats

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
Paul George,0.802571,0.525724,0.248670,0.853365,0.420906,0.704406,0.755930,0.554410,0.553571,4.312411
LeBron James,0.829523,0.526977,0.252095,0.512821,0.687104,0.416046,0.948392,0.503093,0.620671,4.055379
Kemba Walker,0.793573,0.276124,0.180851,0.531674,0.584245,0.723240,0.781568,0.563098,0.455882,3.978491
Nikola Vucevic,0.618022,0.761826,0.383623,0.384615,0.358233,0.242176,0.673814,0.264602,0.380231,3.306681
Marc Gasol,0.490756,0.544161,0.537073,0.524476,0.443387,0.316692,0.305948,0.351811,0.391414,3.122888
Victor Oladipo,0.589786,0.400732,0.144681,0.646154,0.523256,0.442623,0.362031,0.237400,0.450238,2.896423
Robert Covington,0.391813,0.347561,0.431709,0.806452,0.125031,0.489159,0.170436,0.181852,0.244048,2.699966
Nikola Mirotic,0.526235,0.585451,0.241135,0.288462,0.100937,0.512295,0.253712,0.230216,0.215278,2.523163


In [53]:
total.sort_values()

FT%         4.229750
BLK_G       4.815615
AST_G       5.197286
TOV_G       5.371454
FG%         5.907594
3P_G        5.959496
TRB_G       6.266895
PTS_G       7.858320
STL_G       7.945583
sum        42.809086
dtype: float64

### Last 10 Games

In [54]:
data_group_10 = gamelogs[gamelogs['Game_Num'] <= 10]
data_group_10 = data_group_10.groupby(by=["Player"]).agg({'GmSc':'size','PTS':['sum'],'TRB':['sum'],'BLK':['sum'], \
                                                    'STL':['sum'], 'AST':['sum'], '3P':['sum'], 'FG':['sum'], 'FGA':['sum'], \
                                                    'FT':['sum'], 'FTA':['sum'], 'TOV':['sum']})

In [55]:
data_group_10["PTS_G"] = data_group_10["PTS", "sum"] / data_group_10["GmSc", "size"]
data_group_10["TRB_G"] = data_group_10["TRB", "sum"] / data_group_10["GmSc", "size"]
data_group_10["BLK_G"] = data_group_10["BLK", "sum"] / data_group_10["GmSc", "size"]
data_group_10["STL_G"] = data_group_10["STL", "sum"] / data_group_10["GmSc", "size"]
data_group_10["AST_G"] = data_group_10["AST", "sum"] / data_group_10["GmSc", "size"]
data_group_10["3P_G"] = data_group_10["3P", "sum"] / data_group_10["GmSc", "size"]

data_group_10["FGA"] = np.where(data_group_10["FGA"] == 0, 1, data_group_10["FGA"])
data_group_10["FG%"] = data_group_10["FG", "sum"]**2 / data_group_10["GmSc", "size"]
# data_group["FGA", "sum"]

data_group_10["FTA"] = np.where(data_group_10["FTA"] == 0, 1, data_group_10["FTA"])
data_group_10["FT%"] = data_group_10["FT", "sum"]**2 / data_group_10["FTA", "sum"] 
# data_group["FTA", "sum"]

data_group_10["TOV_G"] = data_group_10["TOV", "sum"] / data_group_10["GmSc", "size"]

In [56]:
x_10 = data_group_10[column_names_to_normalize].values
x_scaled_10 = scaler.fit_transform(x_10)
df_temp_10 = pd.DataFrame(x_scaled_10, columns=column_names_to_normalize, index = data_group_10.index)
data_group_10[column_names_to_normalize] = df_temp_10

In [57]:
data_group_10["sum"] = data_group_10["PTS_G"] + data_group_10["TRB_G"] + data_group_10["BLK_G"] + data_group_10["STL_G"] + \
                        data_group_10["AST_G"] + data_group_10["3P_G"] + data_group_10["FG%"] + data_group_10["FT%"] - \
                        data_group_10["TOV_G"] 
data_group_10 = data_group_10.sort_values(by=['sum'], ascending=False)

In [58]:
data_group_10 = data_group_10.loc[:, ['PTS_G', 'TRB_G', 'BLK_G', 'STL_G', 'AST_G', '3P_G', 'FG%', 'FT%', 'TOV_G', 'sum']]

In [59]:
team_stats_10 = data_group_10[data_group_10.index.isin(team["Player"])]
total_10 = team_stats_10.sum(axis=0)

In [60]:
team_stats_10

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
Paul George,0.784264,0.588235,0.096774,0.735294,0.445545,0.690909,0.848338,0.518508,0.60,4.107867
Kemba Walker,0.659898,0.287582,0.258065,0.470588,0.544554,0.727273,0.679902,0.256112,0.42,3.463973
LeBron James,0.624365,0.653595,0.096774,0.382353,0.831683,0.290909,0.637196,0.288126,0.62,3.185001
Nikola Vucevic,0.489848,0.908497,0.322581,0.264706,0.336634,0.218182,0.504848,0.135434,0.38,2.800728
Marc Gasol,0.355330,0.450980,0.483871,0.352941,0.584158,0.218182,0.192367,0.246688,0.56,2.324518
Trevor Ariza,0.352792,0.398693,0.000000,0.705882,0.425743,0.454545,0.177285,0.166336,0.40,2.281276
Kent Bazemore,0.431472,0.274510,0.258065,0.529412,0.386139,0.418182,0.286319,0.169345,0.48,2.273442
Robert Covington,0.360406,0.379085,0.322581,0.500000,0.168317,0.400000,0.177285,0.170256,0.28,2.197930


In [61]:
total_10.sort_values()

FT%         3.130925
BLK_G       3.741935
FG%         4.965682
3P_G        5.163636
AST_G       5.613861
TOV_G       6.020000
STL_G       6.235294
TRB_G       6.398693
PTS_G       6.500000
sum        35.730027
dtype: float64

### Check on New Player

In [62]:
data_group1[(data_group1["sum"] >= min(team_stats["sum"])) & (data_group1["sum"] <= 1.3*min(team_stats["sum"]))]

# Gary Harris (injured)
# JaVale McGee (injured)
# Otto Porter (injured)
# Brook Lopez
# Dennis Schroder
# Nemanja Bjelica
# Kelly Oubre
# Terrence Ross

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
T.J. Warren,0.563353,0.260163,0.268360,0.434243,0.137534,0.396616,0.458241,0.245956,0.232143,2.532322
Jamal Murray,0.560340,0.282582,0.164410,0.361305,0.466878,0.347740,0.469941,0.319047,0.441739,2.530505
Nikola Mirotic,0.526235,0.585451,0.241135,0.288462,0.100937,0.512295,0.253712,0.230216,0.215278,2.523163
Jayson Tatum,0.492063,0.405052,0.248024,0.439560,0.171650,0.357143,0.356509,0.320136,0.289966,2.500172
Buddy Hield,0.594513,0.326160,0.138298,0.203620,0.225148,0.620781,0.533541,0.222908,0.369048,2.495922
Hassan Whiteside,0.388471,0.792102,0.943009,0.233516,0.083056,0.014637,0.235708,0.101457,0.303146,2.488811
D'Angelo Russell,0.531270,0.229280,0.110520,0.427350,0.592162,0.478142,0.484102,0.183042,0.548446,2.487423
Jusuf Nurkic,0.440775,0.646760,0.457447,0.373303,0.276448,0.018081,0.287548,0.382826,0.407038,2.476150


In [63]:
data_group1[data_group1.index == "Tyus Jones"]

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
Tyus Jones,0.137742,0.101925,0.021277,0.452489,0.319197,0.102459,0.030246,0.058816,0.097689,1.126461
